# Merging datasets

In [1]:
# setup
import pandas as pd
import numpy as np

df1 = pd.DataFrame(np.arange(16).reshape(4,4), columns=list("ABCD"))
df2 = pd.DataFrame(np.zeros((3,4)), columns=list("ABCD"))
df1

,A,B,C,D
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


In [2]:
np.zeros([3,4])

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [3]:
df2

,A,B,C,D
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0


## Concat
- concetenates along a particular axis
- set logic

In [4]:
pd.concat([df1, df2])
# df1 and df2 have the same columns name
# index are same as other file

,A,B,C,D
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0
3,12.0,13.0,14.0,15.0
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0


In [5]:
pd.concat([df1, df2]).reset_index(drop=True)

,A,B,C,D
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0
3,12.0,13.0,14.0,15.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0


In [6]:
pd.concat([df1, df2], axis=1)
# df1 shape (4,4), df2 shape (3,4)

,A,B,C,D,A,B,C,D
0,0,1,2,3,0.0,0.0,0.0,0.0
1,4,5,6,7,0.0,0.0,0.0,0.0
2,8,9,10,11,0.0,0.0,0.0,0.0
3,12,13,14,15,NaN,NaN,NaN,NaN


In [7]:
pd.concat([df1, df2], axis=1, join="inner")

,A,B,C,D,A,B,C,D
0,0,1,2,3,0.0,0.0,0.0,0.0
1,4,5,6,7,0.0,0.0,0.0,0.0
2,8,9,10,11,0.0,0.0,0.0,0.0


## Merge

- high performance join operations similar to relational databases
- performance is faster using merge/join in Pandas than in SQL

Relational algebra
- one-to-one - joining 2 dfs on their index
- many-to-one - joining a unique index to ≥ 1 cols in different df
- many-to-many - joining columns on columns

In [8]:
# merge
left = pd.DataFrame({"key": ["K0", "K0", "K1", "K2"],
                     "A": ["A0", "A1", "A2", "A3"],
                     "B": ["B0", "B1", "B2", "B3"],
                     })
right = pd.DataFrame({"key": ["K0", "K1", "K2", "K3"],
                     "C": ["C0", "C1", "C2", "C3"],
                      "D": ["D0", "D1", "D2", "D3"]})
print(left)
print(right)

  key   A   B
0  K0  A0  B0
1  K0  A1  B1
2  K1  A2  B2
3  K2  A3  B3
  key   C   D
0  K0  C0  D0
1  K1  C1  D1
2  K2  C2  D2
3  K3  C3  D3


In [9]:
pd.merge(left, right, on="key", indicator=True) # how= "inner" default

,key,A,B,C,D,_merge
0,K0,A0,B0,C0,D0,both
1,K0,A1,B1,C0,D0,both
2,K1,A2,B2,C1,D1,both
3,K2,A3,B3,C2,D2,both


In [10]:
left.merge(right, on=["key"], how="outer", indicator=True)

,key,A,B,C,D,_merge
0,K0,A0,B0,C0,D0,both
1,K0,A1,B1,C0,D0,both
2,K1,A2,B2,C1,D1,both
3,K2,A3,B3,C2,D2,both
4,K3,NaN,NaN,C3,D3,right_only


In [11]:
left.merge(right, on="key", how="left", indicator=True)

,key,A,B,C,D,_merge
0,K0,A0,B0,C0,D0,both
1,K0,A1,B1,C0,D0,both
2,K1,A2,B2,C1,D1,both
3,K2,A3,B3,C2,D2,both


In [12]:
left = pd.DataFrame(
    {
        "key1": ["K0", "K0", "K1", "K2"],
        "key2": ["K0", "K1", "K0", "K1"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)

right = pd.DataFrame(
    {
        "key1": ["K0", "K1", "K1", "K2"],
        "key2": ["K0", "K0", "K0", "K0"],
        "A": ["A0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

print(left)

print(right)

# only keys present in left
left.merge(right, on=["key1", "key2"], how="left", indicator=True)

  key1 key2   A   B
0   K0   K0  A0  B0
1   K0   K1  A1  B1
2   K1   K0  A2  B2
3   K2   K1  A3  B3
  key1 key2   A   D
0   K0   K0  A0  D0
1   K1   K0  C1  D1
2   K1   K0  C2  D2
3   K2   K0  C3  D3


,key1,key2,A_x,B,A_y,D,_merge
0,K0,K0,A0,B0,A0,D0,both
1,K0,K1,A1,B1,NaN,NaN,left_only
2,K1,K0,A2,B2,C1,D1,both
3,K1,K0,A2,B2,C2,D2,both
4,K2,K1,A3,B3,NaN,NaN,left_only


In [13]:
link = "https://en.wikipedia.org/wiki/List_of_potentially_habitable_exoplanets"
tables = pd.read_html(link)
tables

ImportError: lxml not found, please install it